In [43]:
from subtitle_process import fetch_subtitle
import getpass
import os
import json
import re
from templates import *
from functions import inference

In [44]:
# 设置临时环境变量
def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("bilibili_ssid")

In [45]:
with open("url.json", 'r') as file:
    data = json.load(file)

In [46]:
data[0]

'BV1rK4y1s7er'

In [47]:
problems = []
for bv in data:
    problems.append(fetch_subtitle(bv))

Extracted segment: 1rK4y1s7er
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/883766809211967439303a2c4f792986cb83040badca8845a8?auth_key=1720595992-be4864a427204fb0bd5b7baa72fc5c11-0-c5c90ce361fa514e17c8754743e59d53
Extracted segment: 1344y1Y7sv
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/e347f2f7bd218c7c9a69f739171fc99f?auth_key=1720595995-0038a1a817104865becf550734e6b06c-0-9b453a934bd95618965a9289f246a163
Extracted segment: 15M4y1A7Ru
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/934720128458115642871c4e7162b6c5c313fe6400bff77840?auth_key=1720595998-af7da50cafd641a0a0cf1d912e17c83f-0-b96170a743d379ab0e124fda5f4a6472
Extracted segment: 1Wv4113773
initial url fetch success!
the length of subtitles is: 1
https://aisubtitle.hdslb.com/bfs/ai_subtitle/prod/25062864241261411414244e3a3792bdb60c250477c4261db4?au

In [48]:
def extract_question_explanations(responses):
    questions_and_explanations = []
    for response in responses:
        pattern = re.compile(r"### 原题：\n(.+?)\n\n### 讲解：\n(.+?)\n\n### 相似题一：\n(.+?)\n\n### 相似题一讲解：\n(.+?)(?:\n\n|$)", re.DOTALL)
        matches = pattern.findall(response)

        for match in matches:
            questions_and_explanations.append({
                "original_question": match[0].strip(),
                "original_explanation": match[1].strip(),
                "similar_question_1": match[2].strip(),
                "similar_explanation_1": match[3].strip(),
            })

    with open('casestudy_problem_answer.json', 'w', encoding='utf-8') as f:
        json.dump(questions_and_explanations, f, ensure_ascii=False, indent=4)


In [51]:
template_sub_1_similar = r"""
以下是一段视频字幕，请提取出题目和老师对题目的讲解，同时构造一个相似题目并生成与老师讲解风格相似的讲解。请按如下格式输出：

### 原题：
（提取出的原题）

### 讲解：
（提取出的讲解）

### 相似题一：
（构造的相似题目一）

### 相似题一讲解：
（构造的相似题目一的讲解）

视频字幕：
{subtitle}

注意事项：
1. 对于每一道原题，仅构造一道相似题，对于每一题都当作是新题来讲解，不要出现任何类似于“根据我们刚才说的”这样的话，每一个讲解必须以答案结尾，不能够没有答案。
2. 保留老师讲解风格但是把和讲解无关的内容过滤掉，比如自问自答的部分都要删除掉。
"""

template_expl = r"""
    Role:
        你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一的讲解一个题目。 
    Goals:
        根据给定的板书内容像老师一样讲解板书内容，根据输入决定你正在输出的是讲解几，比如如果输入是板书一，那么输出就是讲解一，以此类推。

    严格按照以下列字符串格式输出:
        ### 讲解一:    ### 讲解二:      ...

    原题和参考信息:
        题目:{question}
        参考信息:{ref}

    注意事项:
        1. 你的讲解要尽量贴合人类老师的讲解风格，言简意赅，清晰明了。
        2. 讲解内容应针对板书的具体步骤进行详细说明，解释每一步的思路和方法。
        3. 确保每一个讲解都对应一个板书。
        4. 板书和讲解的编号必须一一对应，格式必须严格遵守。
        5. 讲解内容应突出解题思路和方法，避免过多冗余信息。
        6. 你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇
        7. 其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题，别用括号1234。
            d: 初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
            e: 对于所有题目尽可能有最简单的方式来进行解答，以最大限度上避免超纲的情况发生。

    下面是板书的内容(请确保每一个讲解都对应一个板书)：
    {notes}

    下面是一些名师讲解的其他题目，请你学习他们的讲解风格，并运用到板书的讲解中(注意：你的讲解对象是一位学生而不是一群学生，一定不要出现同学们这样的复数词汇)。
    注意：初中还没有学过排列组合的问题，避免使用排列组合的知识点解答问题。
    {model_expl}
"""

template_orig = r"""
    Role:
    你是一位专业且循循善诱数学教师名为小派老师,可以解决小学、初中、高中的数学题目,你将一对一讲解一个题目
    Goals:
    根据给定信息按照要求像老师一样讲解一个题目

    技能
    可以使用代码辅助解题
    能够处理各种计算

    Workflows:
    你会按下面的框架来讲解题目
        1. 给出该题目涉及的知识点
        2. 判断给出的信息中是不是包含原题并且有原题的解答过程，如果有原题，板书使用原题的解答步骤，并对应生成讲解；如果没有原题，则参照相似题的思路和解法给出结果

    第一部分:原题和参考信息
        题目:{question}
        学段：初中，你需要适配初中的知识点，解法和书写表达不超纲
        参考信息:{ref}
    第二部分: 以下列字符串格式输出
        知识点讲解:这题涉及到了一些xxx相关的知识点，通常使用xxx方法解决 ### 板书一:     ### 板书二:       ### 讲解一:       ### 讲解二:      ...
    注意事项:
        1.你擅将利用参考信息来解答问题,如果有参考信息，参考信息是这个题目的相似题，你需要参考其解题思路与步骤。
        2.你的解题思路、方法、讲解需要尽可能地参照参考信息,在计算部分使用代码进行辅助，减少自己解题的比重。
        3.知识点讲解部分，需给出题目涉及的知识点考点、题目解题的关键点，必须使用模板，有且只有这一句话就行：本题考查的知识点是.....，解题关键点是......。

        4.板书是这个题目的答题步骤，思路和知识点讲解部分做到一致，按照标准答案的书写方式去写，板书切分块的模式依照 条件+结论为一个单元分块，板书一/二/三/....整个解题步骤不超过十个板书+讲解单元；讲解是对应板书模块的说明。在板书的最开始按需写出"解："或"证明："，在板书最后写出"答："或"故答案是："。
        5.每个式子要独立成行，不输出连续很长的一个式子。
        6.注意板书只写计算公式,尽量少出现中文,一些推导过程涉及到详细的题目条件的要用 依题意得/由题可知 等说法省略详细条件，避免板书中出现很长的中文句子内容，尽量用式子说话, 式子计算过程要做到关键步骤不省略不跳步,如果需要,在一些公式前使用$\because$ $\therefore$等表达因为所以的数学符号
        7.禁止出现**表示标题和-表示列表这种markdown格式的语法，latex公式中避免使用\text{{}}除非必须，适时换行，不要用\\表示换行。
        8.如果题目包含多个小题，板书中对应的开始的地方展示小题题号，比如 (1)，(2)，(3)，禁止重复展示小标题序号，比如板书一板书二都是对于(1)小题的解答，则只需要在板书一开始写一次(1)，板书二不能再写一次(1)。
        9.讲解和板书一一对应，讲解要重点强调对应板书涉及到的思路和方法，讲解要抓住题目重点，言简意赅，语言精炼。  
        例如:
        "知识点讲解:本题考查的知识点是.....，解题关键点是......。### 板书一: $\because$ $\frac{{x^2-2*x-1}}{{x-1}}=\frac{{(x-1)^2}}{{x-1}}=(x-1)^{{2-1}}=x-1$### 板书二: $\therefore$ $\frac{{x^2-2*x-1}}{{x-1}}*(x-2)=(x-1)(x-2)=x^2-3*x+2$ \n故答案是$x^2-3*x+2$### 讲解一:首先利用完全平方公式将分子化简,随后发现分子分母都是(x-1)的次幂,因此将指数相减即可。### 讲解二:根据上述化简结果,代入原式即可得到最终答案$x^2-3*x+2$"
        10.返回的字段中只能出现知识点讲解、板书N、讲解N(N代表中文里的一、二、三等正整数)，且所有的板书模块出完了才能出讲解模块，板书模块数量和讲解模块数量必须一致，内容必须一一对应...不能出任何现其他的字段，在最后一个板书部分要有总结性话语，在这个例子中板书二是最后一个板书部分，所以最后一句话说 故答案是...，你的回答中最后一个板书n的部分中，最后一句话必须是，故答案是...
        11.板书参考书写范式，你需要根据题型参照范式输出板书：
            {ref_form}
        12.其它注意事项：
            a: 初中不等式还没有学到用方括号圆括号以及并集交集等表示范围，避免使用这些表达。
            b: 初中几何题目如证明全等三角形，正面直线平行，板书中需要写上使用了什么定理，并且注意写法是定理短句加括号写在式子后面。
            c: 对于选择题的问题，如果需要把各个选项铺陈，用ABCD作为列表标题别用括号1234。
            d: 对于所有题目尽可能用解释性的语言进行解答，不到万不得已不用公式直接求解，以最大限度上避免超纲的情况发生。
        13.板书不能超过10条，如果10条写不下，使用别的方法来解题。
"""

In [52]:
raw_problem_answer = []
for problem in problems:
    raw_problem_answer.append(await inference({"subtitle": problem}, template_sub_1_similar))


executing
### 原题：
已知三角形ABC中，AB等于AC等于BD，角BAC等于90度，角ABD等于30度。求证AD等于CD。

### 讲解：
这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且BD也等于它的腰，ABD是30度。我们需要证明AD等于DC。

要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角ABD是30度，由于它是等腰直角三角形，所以角DBC是15度。这个等腰三角形的顶角是30度，两个底角分别是75度。

我们在BC上截取一个BE，使得BE等于AD，然后连接DE。显然，三角形ADC和三角形BED全等。因为AC等于BD，角CAD等于角DBE，AD等于BE，所以三角形ADC全等于三角形BED（SAS）。全等之后，我们可以得到ED等于DC。

接下来，我们利用ED等于DC这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角EDE等于15度加上一个角X，另一边的角等于45度减X。因为这两个角相等，所以15度加X等于45度减X，解得X等于15度。这样，ADC是一个等腰三角形，AD等于DC。

### 相似题一：
已知三角形PQR中，PQ等于PR等于PS，角PQR等于90度，角PQS等于45度。求证PS等于QS。

### 相似题一讲解：
这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且PS也等于它的腰，PQS是45度。我们需要证明PS等于QS。

要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角PQS是45度，由于它是等腰直角三角形，所以角QSR是45度。这个等腰三角形的顶角是45度，两个底角分别是45度。

我们在QR上截取一个QT，使得QT等于PS，然后连接TS。显然，三角形PQS和三角形QTS全等。因为PQ等于PR，角PQS等于角QTS，PS等于QT，所以三角形PQS全等于三角形QTS（SAS）。全等之后，我们可以得到TS等于QS。

接下来，我们利用TS等于QS这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角TQS等于45度加上一个角Y，另一边的角等于45度减Y。因为这两个角相等，所以45度加Y等于45度减Y

In [53]:
raw_problem_answer

['### 原题：\n已知三角形ABC中，AB等于AC等于BD，角BAC等于90度，角ABD等于30度。求证AD等于CD。\n\n### 讲解：\n这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且BD也等于它的腰，ABD是30度。我们需要证明AD等于DC。\n\n要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角ABD是30度，由于它是等腰直角三角形，所以角DBC是15度。这个等腰三角形的顶角是30度，两个底角分别是75度。\n\n我们在BC上截取一个BE，使得BE等于AD，然后连接DE。显然，三角形ADC和三角形BED全等。因为AC等于BD，角CAD等于角DBE，AD等于BE，所以三角形ADC全等于三角形BED（SAS）。全等之后，我们可以得到ED等于DC。\n\n接下来，我们利用ED等于DC这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角EDE等于15度加上一个角X，另一边的角等于45度减X。因为这两个角相等，所以15度加X等于45度减X，解得X等于15度。这样，ADC是一个等腰三角形，AD等于DC。\n\n### 相似题一：\n已知三角形PQR中，PQ等于PR等于PS，角PQR等于90度，角PQS等于45度。求证PS等于QS。\n\n### 相似题一讲解：\n这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且PS也等于它的腰，PQS是45度。我们需要证明PS等于QS。\n\n要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角PQS是45度，由于它是等腰直角三角形，所以角QSR是45度。这个等腰三角形的顶角是45度，两个底角分别是45度。\n\n我们在QR上截取一个QT，使得QT等于PS，然后连接TS。显然，三角形PQS和三角形QTS全等。因为PQ等于PR，角PQS等于角QTS，PS等于QT，所以三角形PQS全等于三角形QTS（SAS）。全等之后，我们可以得到TS等于QS。\n\n接下来，我们利用TS等于QS这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角TQS等于45度加上一个角Y，另一边的角等于45度减Y。因为这两个角相等，

In [54]:
extract_question_explanations(raw_problem_answer)

In [55]:
with open("casestudy_problem_answer.json", 'r') as file:
    data = json.load(file)

data

[{'original_question': '已知三角形ABC中，AB等于AC等于BD，角BAC等于90度，角ABD等于30度。求证AD等于CD。',
  'original_explanation': '这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且BD也等于它的腰，ABD是30度。我们需要证明AD等于DC。\n\n要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角ABD是30度，由于它是等腰直角三角形，所以角DBC是15度。这个等腰三角形的顶角是30度，两个底角分别是75度。\n\n我们在BC上截取一个BE，使得BE等于AD，然后连接DE。显然，三角形ADC和三角形BED全等。因为AC等于BD，角CAD等于角DBE，AD等于BE，所以三角形ADC全等于三角形BED（SAS）。全等之后，我们可以得到ED等于DC。\n\n接下来，我们利用ED等于DC这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角EDE等于15度加上一个角X，另一边的角等于45度减X。因为这两个角相等，所以15度加X等于45度减X，解得X等于15度。这样，ADC是一个等腰三角形，AD等于DC。',
  'similar_question_1': '已知三角形PQR中，PQ等于PR等于PS，角PQR等于90度，角PQS等于45度。求证PS等于QS。',
  'similar_explanation_1': '这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且PS也等于它的腰，PQS是45度。我们需要证明PS等于QS。'},
 {'original_question': '已知 \\( BD = 2 \\times DC \\)，\\( E \\) 为 \\( AD \\) 的中点。三角形 \\( ABC \\) 的面积是 30，求四边形 \\( DCF \\) 的面积。',
  'original_explanation': '首先，我们知道 \\( BD = 2 \\times DC \\)，所以如果 \\( DC = x \\)，那么 \\( BD = 2x \\)。由于 \\( E \\) 是 \\( AD \\) 的中点，所以 \\( AE = ED \\)。\n

In [56]:
model_explanation = ""
for ps in data[:3]:
    model_explanation += "题目:\n" + ps['original_question'] + '\n\n'
    model_explanation += "讲解:\n" + ps['original_explanation'] + '\n\n'
    # model_explanation += "题目:\n" + ps['similar_question_1'] + '\n\n'
    # model_explanation += "讲解:\n" + ps['similar_explanation_1'] + '\n\n'
model_explanation

'题目:\n已知三角形ABC中，AB等于AC等于BD，角BAC等于90度，角ABD等于30度。求证AD等于CD。\n\n讲解:\n这是一道证明题，条件非常简单，这是一个等腰直角三角形，并且BD也等于它的腰，ABD是30度。我们需要证明AD等于DC。\n\n要想证明两个线段相等，常规的办法是证明全等三角形。在这个图当中没有全等三角形，所以我们需要构造全等三角形。我们从已知条件出发分析，如果角ABD是30度，由于它是等腰直角三角形，所以角DBC是15度。这个等腰三角形的顶角是30度，两个底角分别是75度。\n\n我们在BC上截取一个BE，使得BE等于AD，然后连接DE。显然，三角形ADC和三角形BED全等。因为AC等于BD，角CAD等于角DBE，AD等于BE，所以三角形ADC全等于三角形BED（SAS）。全等之后，我们可以得到ED等于DC。\n\n接下来，我们利用ED等于DC这个条件，说明这是一个等腰三角形。等腰三角形的两个底角相等，我们可以设角EDE等于15度加上一个角X，另一边的角等于45度减X。因为这两个角相等，所以15度加X等于45度减X，解得X等于15度。这样，ADC是一个等腰三角形，AD等于DC。\n\n题目:\n已知 \\( BD = 2 \\times DC \\)，\\( E \\) 为 \\( AD \\) 的中点。三角形 \\( ABC \\) 的面积是 30，求四边形 \\( DCF \\) 的面积。\n\n讲解:\n首先，我们知道 \\( BD = 2 \\times DC \\)，所以如果 \\( DC = x \\)，那么 \\( BD = 2x \\)。由于 \\( E \\) 是 \\( AD \\) 的中点，所以 \\( AE = ED \\)。\n\n接下来，我们利用三角形的面积比例关系。因为 \\( BD \\) 和 \\( DC \\) 的比是 2:1，所以三角形 \\( ABD \\) 和三角形 \\( ABC \\) 的面积比也是 2:1。总面积是 30，那么三角形 \\( ABD \\) 的面积是 20，三角形 \\( ABC \\) 的面积是 10。\n\n由于 \\( E \\) 是中点，所以 \\( BE \\) 是中线，中线平分面积，因此三角形 \\( ABE \\) 和三角形 \\( EBD \\) 的面积相等，

In [57]:
orig_answers = []
orig_notes = []
orig_expls = []
refined_explanation = []
questions = []

In [58]:
'''
对于每一个<原题,相似题>，用两个prompt来分别对相似题进行解答
    1. 用original prompt对相似题直接进行解答
    2. 将原题解答及相似题解答作为few shots examples喂给另一个prompt来对比两种的效果哪一个比较好
'''

with open("casestudy_problem_answer.json", 'r') as file:
    problem_set = json.load(file)


for ps in problem_set:
    orig_answers.append(await inference({"question": ps['similar_question_1'], "ref_form": ref_form, "ref": ref}, template_orig, max_tokens=2000))
    pattern1 = r"### 板书一:[\s\S]*?(?:### 板书[二三四五六七八九十]+:[\s\S]*?)*?(?=### 讲解一)"
    orig_note = re.search(pattern1, orig_answers[-1]).group()
    pattern2 = r"### 讲解一:[\s\S]*"
    orig_expl = re.search(pattern2, orig_answers[-1]).group()
    orig_notes.append(orig_note)
    orig_expls.append(orig_expl)
    questions.append(ps['similar_question_1'])
    refined_explanation.append(await inference({"question": ps['similar_question_1'], "ref": ref, "notes": orig_note, "model_expl": model_explanation}, template_expl, max_tokens=2000))

# print(orig_notes)
# print(refined_explanation)    

executing
知识点讲解:本题考查的知识点是等腰直角三角形的性质和全等三角形的判定，解题关键点是利用等腰直角三角形的性质和全等三角形的判定来证明PS等于QS。

### 板书一:
$\because PQ = PR = PS$
$\angle PQR = 90^\circ$
$\angle PQS = 45^\circ$

### 板书二:
$\therefore \triangle PQS$ 是等腰直角三角形
$PS = PQ$

### 板书三:
$\angle PQS = \angle PSQ = 45^\circ$ (等腰直角三角形的性质)
$\angle PQS + \angle PSQ = 90^\circ$

### 板书四:
$\triangle PQS \cong \triangle QSP$ (SAS全等条件)
$QS = PS$

### 讲解一:
首先，根据题意，PQ、PR和PS相等，且$\angle PQR = 90^\circ$，$\angle PQS = 45^\circ$。这说明$\triangle PQS$是一个等腰直角三角形。

### 讲解二:
由于$\triangle PQS$是等腰直角三角形，所以PS等于PQ。

### 讲解三:
在等腰直角三角形中，两个锐角相等，且每个锐角为45度。因此，$\angle PQS = \angle PSQ = 45^\circ$，并且两个角的和为90度。

### 讲解四:
根据SAS全等条件，$\triangle PQS$和$\triangle QSP$全等，因此可以得出QS等于PS。故答案是PS等于QS。executing
### 讲解一:
首先，我们来看题目给出的条件：
- \( PQ = PR = PS \)
- \( \angle PQR = 90^\circ \)
- \( \angle PQS = 45^\circ \)

这些条件告诉我们，三角形 \( PQR \) 是一个等腰直角三角形，因为 \( PQ \) 和 \( PR \) 相等，并且 \( \angle PQR \) 是直角。

### 讲解二:
根据前面的条件，我们可以得出三角形 \( PQS \) 是一个等腰直角三角形。因为在等腰直角三角形中，两条直角边相等，所以 \( PS = PQ \)。

### 

In [59]:
with open("compare_results_p8.md", 'a') as file:
    for i in range(len(questions)):
        file.write("# 原题：\n" + questions[i] + '\n\n')
        file.write("# 原本板书：\n" + orig_notes[i] + '\n\n')
        file.write("# 原本解答：\n" + orig_expls[i] + '\n\n')
        file.write("# 新解答：\n" + refined_explanation[i] + '\n\n')

In [60]:
print(len(refined_explanation))

4


In [32]:
import re

text = """
知识点讲解:本题考查的知识点是因式分解与方程求解，解题关键点是利用平方差公式将原式化简并求解。

###板书一:
解：
$(2a+2b+1)(2a+2b-1)=63$

$设x=2a+2b$

$则(x+1)(x-1)=63$

$x^2-1=63$

$x^2=64$

$x=±8$

$2a+2b=8$

$a+b=4$

$2a+2b=-8$

$a+b=-4$

故答案是$a+b=4$或$a+b=-4$

###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。
"""

pattern = r"###讲解一:[\s\S]*"
match = re.search(pattern, text).group()

if match:
    print(match)
else:
    print("No match found")


###讲解一:
首先，我们将原式$(2a+2b+1)(2a+2b-1)=63$进行因式分解，利用平方差公式$(x+1)(x-1)=x^2-1$，设$x=2a+2b$，则原式变为$x^2-1=63$。

###讲解二:
接下来，我们解方程$x^2-1=63$，得到$x^2=64$，所以$x=±8$。然后将$x$代入$2a+2b=x$，分别得到$2a+2b=8$和$2a+2b=-8$，从而求得$a+b=4$或$a+b=-4$。故答案是$a+b=4$或$a+b=-4$。

